In [46]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ruptures as rpt
from scipy.io import loadmat, savemat
from scipy.signal import butter, lfilter, filtfilt
from scipy.stats import multivariate_normal
import scipy.optimize as optimize

%matplotlib notebook

In [63]:
def tkeo(signal):
    tkeo_signals = []
    for i in range(1, len(signal)-1):
        tkeo_signals += [(signal[i])**2 - signal[i-1]*signal[i+1]]
    return tkeo_signals


#Band pass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y


#High pass filter
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

#Low pass filter
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y


# Using idea from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2945630/
def process(signal):
    signal = butter_bandpass_filter(signal, 30, 200, 1000)
    signal = tkeo(signal)
    signal = np.absolute(signal)
    signal = butter_lowpass_filter(signal, 50, 1000)
    return signal


In [64]:
directory = '/global/scratch/zfhusted/realtime_experiments/'
files = os.listdir(directory)

# experiment
# [0, session index]
# [1]
# [0, gesture index]
# [2]
#signal = loadmat(annot)['experiment']

In [65]:
signal_dfs = {}
for file in files:
    signal = loadmat(directory + file)['experiment']
    df = pd.DataFrame(columns=["session_id", "gesture_id", "channel", "raw_data", "processed_data", "onset", "offset"])
    for i in range(signal[0].shape[0]):
        for j in range(signal[0,i][1][0].shape[0]):
            for k in range(len(np.transpose(signal[0,i][1][0,j][2]))):
                df = df.append({'session_id': i, 'gesture_id': j, "channel": k, 'raw_data': np.transpose(signal[0,i][1][0,j][2])[k]}, ignore_index=True)
    signal_dfs[file] = df

In [66]:
for file in files:
    for s in range(len(signal_dfs[file])):
        processed = process(signal_dfs[file].loc[s].raw_data)
        signal_dfs[file].loc[s].processed_data = processed

/global/software/sl-7.x86_64/modules/langs/python/3.6/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [67]:
files

['s1_mu3.mat',
 's1_ap2.mat',
 's1_ap3.mat',
 's1_ap1.mat',
 's1_pw1.mat',
 's2_pw1.mat',
 's1_mu1.mat',
 's2_nd3.mat',
 's2_ap3.mat',
 's2_pw2.mat',
 's2_pw3.mat',
 's2_mu3.mat',
 's1_pw3.mat',
 's1_nd2.mat',
 's2_ap2.mat',
 's1_mu2.mat',
 's2_mu2.mat',
 's1_nd1.mat',
 's2_mu1.mat',
 's2_nd2.mat',
 's1_pw2.mat',
 's2_nd1.mat',
 's2_ap1.mat',
 's1_nd3.mat']

In [68]:
signal_dfs

{'s1_ap1.mat':      session_id gesture_id channel  \
 0             0          0       0   
 1             0          0       1   
 2             0          0       2   
 3             0          0       3   
 4             0          0       4   
 5             0          0       5   
 6             0          0       6   
 7             0          0       7   
 8             0          0       8   
 9             0          0       9   
 10            0          0      10   
 11            0          0      11   
 12            0          0      12   
 13            0          0      13   
 14            0          0      14   
 15            0          0      15   
 16            0          0      16   
 17            0          0      17   
 18            0          0      18   
 19            0          0      19   
 20            0          0      20   
 21            0          0      21   
 22            0          0      22   
 23            0          0      23   
 24        

In [69]:
for file in files:
    for s in range(len(signal_dfs[file])):
        #Changepoint detection with the Binary Segmentation search method & gaussian loss function
        model = "normal"  
        algo = rpt.Binseg(model=model).fit(signal_dfs[file].loc[s].processed_data[0:6000])
        my_bkps = algo.predict(n_bkps=1)
        signal_dfs[file].loc[s].onset = my_bkps[0]

In [70]:
for file in files:
    for s in range(len(signal_dfs[file])):
        #Changepoint detection with the Binary Segmentation search method & gaussian loss function
        model = "normal"  
        algo = rpt.Binseg(model=model).fit(signal_dfs[file].loc[s].processed_data[3000:])
        my_bkps = algo.predict(n_bkps=1)
        signal_dfs[file].loc[s].offset = my_bkps[0]

In [71]:
signal_dfs

{'s1_ap1.mat':      session_id gesture_id channel  \
 0             0          0       0   
 1             0          0       1   
 2             0          0       2   
 3             0          0       3   
 4             0          0       4   
 5             0          0       5   
 6             0          0       6   
 7             0          0       7   
 8             0          0       8   
 9             0          0       9   
 10            0          0      10   
 11            0          0      11   
 12            0          0      12   
 13            0          0      13   
 14            0          0      14   
 15            0          0      15   
 16            0          0      16   
 17            0          0      17   
 18            0          0      18   
 19            0          0      19   
 20            0          0      20   
 21            0          0      21   
 22            0          0      22   
 23            0          0      23   
 24        

In [62]:
for file in files:
    grouped = signal_dfs[file].groupby(['session_id', 'gesture_id']).agg(list)
    grouped.to_csv('realtime_experiments_breakpoints_grouped/' + file[:-4], index=False)

In [72]:
signal_dfs

{'s1_ap1.mat':      session_id gesture_id channel  \
 0             0          0       0   
 1             0          0       1   
 2             0          0       2   
 3             0          0       3   
 4             0          0       4   
 5             0          0       5   
 6             0          0       6   
 7             0          0       7   
 8             0          0       8   
 9             0          0       9   
 10            0          0      10   
 11            0          0      11   
 12            0          0      12   
 13            0          0      13   
 14            0          0      14   
 15            0          0      15   
 16            0          0      16   
 17            0          0      17   
 18            0          0      18   
 19            0          0      19   
 20            0          0      20   
 21            0          0      21   
 22            0          0      22   
 23            0          0      23   
 24        

array([(array([[None]], dtype=object), array([[None]], dtype=object), array([[None]], dtype=object), array([[None]], dtype=object), array([[None]], dtype=object), array([[None]], dtype=object), array([[None]], dtype=object))],
      dtype=[('session_id', 'O'), ('gesture_id', 'O'), ('channel', 'O'), ('raw_data', 'O'), ('processed_data', 'O'), ('onset', 'O'), ('offset', 'O')])